***SCRAP ANIMELIST***

In [1]:
import requests
import lxml.html as html
import pandas as pd
import numpy as np

from collections import defaultdict

In [2]:
url_padre = "https://myanimelist.net/anime/season"

In [3]:
#Obteniendo titulos y score de estrenos animes de temporada
temporada = '//h1//a[@class="on"]/text()'
titulo_temporada = '//div[2]/div[3]/div[1]//a[@class="link-title"]/text()'
score_temporada = '//div[2]/div[3]/div[1]//div[@title="Score"]/text()'
imagenes_temporada = '//div[2]/div[3]/div[1]//div[@class="image"]//a//@src'
#Obteniendo titulos de animes en emisión.
# titulo_emision =  '//div[@class="product_price"]/p[@class="price_color"]/text()'

#Obteniendo Categoria
# categoria = '//div[@class="page-header action"]/h1/text()'

In [4]:
#requests
r = requests.get(url_padre)
print(r.status_code)

200


#### __RESPUESTA HTTP__

* Respuestas informativas (100–199),  
* Respuestas satisfactorias (200–299),  
* Redirecciones (300–399),  
* Errores de los clientes (400–499),  
* y errores de los servidores (500–599).  

*REQUEST MANUALES DE PRUEBA*

In [5]:
#requests

r = requests.get(url_padre)

home = r.content.decode("utf-8")

parser = html.fromstring(home)

#Temporada 
temporada_analisis = parser.xpath(temporada)
temporada_analisis = list(map(lambda x: x.replace('\n', ''), temporada_analisis))
temporada_analisis = [x.strip() for x in temporada_analisis]
temporada_analisis

#Titulos anime temporada
titulos_anime_streno_temporada = parser.xpath(titulo_temporada)

#Score anime temporada
score_anime_streno_temporada = parser.xpath(score_temporada)
score_anime_streno_temporada = [x for x in score_anime_streno_temporada if x != '\n          ']
score_anime_streno_temporada = list(map(lambda x: x.replace('\n        ', ''), score_anime_streno_temporada))

#Imagenes animes
# imagenes_anime_temporada = parser.xpath(imagenes_temporada)

**Rutina para obtener imagenes***

In [6]:
#Imagenes animes

from lxml import html 
imagenes_dic = []
tree = html.fromstring(home)
for link_element in tree.xpath('//div[2]/div[3]/div[1]//div[@class="image"]//a/img'):
    # href1 = link_element.get('src')
    # print(type(href1))
    if (link_element.get('src') is None):
        href1 = link_element.get('data-src')
        imagenes_dic.append(href1)
        # print(href1)
    else:
        href1 = link_element.get('src')
        imagenes_dic.append(href1)
        # print(href1)

len(imagenes_dic)

54

__FUNCIÓN ANIME TEMPORADA__

In [7]:
def parser_anime_temporada(url): 
    content_dic = {}
    r = requests.get(url)
    home = r.content.decode("utf-8")
    parser = html.fromstring(home)   

    if r.status_code == 200:
        
        #OBTENIENDO TITULOS Y PRECIO
        
        titulo_temporada = '//div[2]/div[3]/div[1]//a[@class="link-title"]/text()'
        titulos_anime_streno_temporada = parser.xpath(titulo_temporada)
        content_dic['Anime Season'] = titulos_anime_streno_temporada

        score_temporada = '//div[2]/div[3]/div[1]//div[@title="Score"]/text()'
        score_anime_streno_temporada = parser.xpath(score_temporada)
        score_anime_streno_temporada = [x for x in score_anime_streno_temporada if x != '\n          ']
        score_anime_streno_temporada = list(map(lambda x: x.replace('\n        ', ''), score_anime_streno_temporada))
        content_dic['Score Anime Season'] = score_anime_streno_temporada

        temporada = '//h1//a[@class="on"]/text()'
        temporada_analisis = parser.xpath(temporada)
        temporada_analisis = list(map(lambda x: x.replace('\n', ''), temporada_analisis))
        temporada_analisis = [x.strip() for x in temporada_analisis]
        
    imagenes_temporada = []
    # tree = html.fromstring(home)
    for link_element in parser.xpath('//div[2]/div[3]/div[1]//div[@class="image"]//a/img'):
        # href1 = link_element.get('src')
        # print(type(href1))
        if (link_element.get('src') is None):
            href1 = link_element.get('data-src')
            imagenes_temporada.append(href1)
            # print(href1)
        else:
            href1 = link_element.get('src')
            imagenes_temporada.append(href1)
        # print(href1)

    content_dic['Imagenes'] = []
    content_dic.update({"Imagenes": imagenes_temporada})


    list_temporada = []
    for j in range(len(content_dic['Anime Season'])):
        
        list_temporada.append(temporada_analisis)

    content_dic['Temporada'] = []
    content_dic.update({"Temporada": list_temporada})

    return content_dic 

In [8]:
data = []
data.append(parser_anime_temporada(url_padre))

__Creando Dataframe__

In [9]:
df = pd.DataFrame()
for j in data:
    df_uno = pd.DataFrame(j)
    df = pd.concat([df, df_uno])

df

,Anime Season,Score Anime Season,Imagenes,Temporada
0,Spy x Family,9.08,https://cdn.myanimelist.net/images/anime/1441/...,[Spring 2022]
1,Tate no Yuusha no Nariagari Season 2,7.33,https://cdn.myanimelist.net/images/anime/1143/...,[Spring 2022]
2,Kaguya-sama wa Kokurasetai: Ultra Romantic,8.96,https://cdn.myanimelist.net/images/anime/1160/...,[Spring 2022]
3,Kawaii dake ja Nai Shikimori-san,7.23,https://cdn.myanimelist.net/images/anime/1995/...,[Spring 2022]
4,"Komi-san wa, Comyushou desu. 2nd Season",8.32,https://cdn.myanimelist.net/images/anime/1108/...,[Spring 2022]
5,Aharen-san wa Hakarenai,7.49,https://cdn.myanimelist.net/images/anime/1612/...,[Spring 2022]
6,Date A Live IV,8.01,https://cdn.myanimelist.net/images/anime/1368/...,[Spring 2022]
7,"Yuusha, Yamemasu",7.09,https://cdn.myanimelist.net/images/anime/1934/...,[Spring 2022]
8,Tomodachi Game,7.23,https://cdn.myanimelist.net/images/anime/1247/...,[Spring 2022]
9,"Gaikotsu Kishi-sama, Tadaima Isekai e Odekakechuu",7.32,https://cdn.myanimelist.net/images/anime/1361/...,[Spring 2022]


In [13]:
from IPython.core.display import HTML

def path_html_img(url):
    return '<img src="' + url + '"width = "60">'

In [14]:
df['Imagen_vista'] = df['Imagenes'].apply(lambda x : path_html_img(x))
df.head()

,Anime Season,Score Anime Season,Imagenes,Temporada,Imagen_vista
0,Spy x Family,9.08,https://cdn.myanimelist.net/images/anime/1441/...,[Spring 2022],"<img src=""https://cdn.myanimelist.net/images/a..."
1,Tate no Yuusha no Nariagari Season 2,7.33,https://cdn.myanimelist.net/images/anime/1143/...,[Spring 2022],"<img src=""https://cdn.myanimelist.net/images/a..."
2,Kaguya-sama wa Kokurasetai: Ultra Romantic,8.96,https://cdn.myanimelist.net/images/anime/1160/...,[Spring 2022],"<img src=""https://cdn.myanimelist.net/images/a..."
3,Kawaii dake ja Nai Shikimori-san,7.23,https://cdn.myanimelist.net/images/anime/1995/...,[Spring 2022],"<img src=""https://cdn.myanimelist.net/images/a..."
4,"Komi-san wa, Comyushou desu. 2nd Season",8.32,https://cdn.myanimelist.net/images/anime/1108/...,[Spring 2022],"<img src=""https://cdn.myanimelist.net/images/a..."


In [ ]:
# df.to_html(escape=False, formatters=dict(Portada = path_html_img))

In [16]:
HTML(df.to_html(escape=False, formatters=dict(Portada = path_html_img)))

,Anime Season,Score Anime Season,Imagenes,Temporada,Imagen_vista
0,Spy x Family,9.08,https://cdn.myanimelist.net/images/anime/1441/122795.jpg,[Spring 2022],
1,Tate no Yuusha no Nariagari Season 2,7.33,https://cdn.myanimelist.net/images/anime/1143/121873.jpg,[Spring 2022],
2,Kaguya-sama wa Kokurasetai: Ultra Romantic,8.96,https://cdn.myanimelist.net/images/anime/1160/122627.jpg,[Spring 2022],
3,Kawaii dake ja Nai Shikimori-san,7.23,https://cdn.myanimelist.net/images/anime/1995/121695.jpg,[Spring 2022],
4,"Komi-san wa, Comyushou desu. 2nd Season",8.32,https://cdn.myanimelist.net/images/anime/1108/121157.jpg,[Spring 2022],
5,Aharen-san wa Hakarenai,7.49,https://cdn.myanimelist.net/images/anime/1612/120636.jpg,[Spring 2022],
6,Date A Live IV,8.01,https://cdn.myanimelist.net/images/anime/1368/121281.jpg,[Spring 2022],
7,"Yuusha, Yamemasu",7.09,https://cdn.myanimelist.net/images/anime/1934/122301.jpg,[Spring 2022],
8,Tomodachi Game,7.23,https://cdn.myanimelist.net/images/anime/1247/121345.jpg,[Spring 2022],
9,"Gaikotsu Kishi-sama, Tadaima Isekai e Odekakechuu",7.32,https://cdn.myanimelist.net/images/anime/1361/120706.jpg,[Spring 2022],
